1. MODELO SDCNet


In [1]:
#paper:
#https://arxiv.org/pdf/1908.06473v1.pdf
.

In [9]:
cd '/content/drive/My Drive/share/crowd_counting/S-DCN/' 

/content/drive/My Drive/share/crowd_counting/S-DCN


In [2]:
ls

'crowd_ S-DCNet_Official.ipynb'   main_process.py   README.md          Val.py
 IOtools.py                       model/            requirements.txt
 LICENSE                          Network/          SHAB_main.py
 load_data_V2.py                  __pycache__/      Test_Data/


In [3]:
import os
import numpy as np 
from main_process import main
from IOtools import txt_write

    
opt = dict()

dataset_list = {0:'SH_partA_Density_map',1:'SH_partB_Density_map'}
model_list = {0:'model/SHA',1:'model/SHB'}
max_num_list = {0:22,1:7}

# step1: Create root path for dataset
opt['num_workers'] = 0

opt['IF_savemem_test'] = False
opt['test_batch_size'] = 1

# --Network settinng    
opt['psize'],opt['pstride'] = 64,64


# -- start testing
set_len = len(dataset_list)

for ti in range(set_len):
    opt['dataset'] = dataset_list[ti]
    opt['trained_model_path'] = model_list[ti]
    opt['root_dir'] = os.path.join(r'Test_Data',opt['dataset'])

    #-- set the max number and partition
    opt['max_num'] = max_num_list[ti]  
    partition_method = {0:'one_linear',1:'two_linear'}
    opt['partition'] = partition_method[1]
    opt['step'] = 0.5

    print('=='*36)
    print('Begin to test for %s' %(dataset_list[ti]) )
    main(opt)

Begin to test for SH_partA_Density_map


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth



Test:[    1/  182] pre: 206.730 gt:172.000 err:34.730 frame: 0.08Hz/0.08Hz
Test:[    2/  182] pre: 466.982 gt:502.000 err:-35.018 frame: 0.31Hz/0.19Hz
Test:[    3/  182] pre: 394.770 gt:391.000 err:3.770 frame: 0.23Hz/0.21Hz
Test:[    4/  182] pre: 226.043 gt:211.000 err:15.043 frame: 0.43Hz/0.26Hz
Test:[    5/  182] pre: 213.320 gt:223.000 err:-9.680 frame: 0.48Hz/0.30Hz
Test:[    6/  182] pre: 487.490 gt:431.000 err:56.490 frame: 0.28Hz/0.30Hz
Test:[    7/  182] pre: 1149.914 gt:1175.000 err:-25.086 frame: 0.38Hz/0.31Hz
Test:[    8/  182] pre: 206.439 gt:265.000 err:-58.561 frame: 0.42Hz/0.32Hz
Test:[    9/  182] pre: 1005.051 gt:1232.000 err:-226.949 frame: 0.38Hz/0.33Hz
Test:[   10/  182] pre: 180.716 gt:294.000 err:-113.284 frame: 0.46Hz/0.34Hz
Test:[   11/  182] pre: 182.506 gt:182.000 err:0.506 frame: 0.43Hz/0.35Hz
Test:[   12/  182] pre: 328.422 gt:379.000 err:-50.578 frame: 0.40Hz/0.36Hz
Test:[   13/  182] pre: 1013.247 gt:1068.000 err:-54.753 frame: 0.39Hz/0.36Hz
Test:[   14

In [17]:
cd '/content/drive/My Drive/share/crowd_counting/S-DCN/' 

/content/drive/My Drive/share/crowd_counting/S-DCN


2. CASO DE USO, AGLOMERACIÓN EN EL AEROPUERTO JORGE CHAVEZ


In [21]:
import os
from IOtools import txt_write
from load_data_V2 import myDataset, ToTensor
from Network.SDCNet import SDCNet_VGG16_classify
    
opt = dict()

dataset_list = {0:'SH_partA_Density_map',1:'SH_partB_Density_map'}
model_list = {0:'model/SHA',1:'model/SHB'}
max_num_list = {0:22,1:7}

# step1: Create root path for dataset
opt['num_workers'] = 0
opt['IF_savemem_test'] = False
opt['test_batch_size'] = 1
# --Network settinng    
opt['psize'],opt['pstride'] = 64,64



opt['dataset'] = dataset_list[0]
opt['trained_model_path'] = model_list[0]
opt['root_dir'] = os.path.join(r'Test_Data',opt['dataset'])

#-- set the max number and partition
opt['max_num'] = max_num_list[0]  
partition_method = {0:'one_linear',1:'two_linear'}
opt['partition'] = partition_method[1]
opt['step'] = 0.5

#evaluate an imagen
# test the exist trained model
mod_path='best_epoch.pth' 
mod_path=os.path.join(opt['trained_model_path'],mod_path)


# set label_indice
if opt['partition'] =='one_linear':
    label_indice = np.arange(opt['step'],opt['max_num']+opt['step']/2,opt['step'])
    add = np.array([1e-6])
    label_indice = np.concatenate( (add,label_indice) )
elif opt['partition'] =='two_linear':
    label_indice = np.arange(opt['step'],opt['max_num']+opt['step']/2,opt['step'])
    add = np.array([1e-6,0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45]) 
    label_indice = np.concatenate( (add,label_indice) )
# print(label_indice)

#init networks
label_indice = torch.Tensor(label_indice)
class_num = len(label_indice)+1
div_times = 2

net = SDCNet_VGG16_classify(class_num,label_indice,psize=opt['psize'],\
        pstride = opt['pstride'],div_times=div_times,load_weights=True).cuda()


#load model

all_state_dict = torch.load(mod_path)
net.load_state_dict(all_state_dict['net_state_dict'])
log_save_path = os.path.join(opt['trained_model_path'],'log-trained-model.txt')
  # test




In [52]:
dataset = opt['dataset']
root_dir = opt['root_dir']
num_workers = opt['num_workers']
img_subsubdir = 'images'; tar_subsubdir = 'gtdens'
dataset_transform = ToTensor()

#test settings
img_dir = os.path.join(root_dir,'test',img_subsubdir)
tar_dir = os.path.join(root_dir,'test',tar_subsubdir)
rgb_dir = os.path.join(root_dir,'rgbstate.mat')
testset = myDataset(img_dir,tar_dir,rgb_dir,transform=dataset_transform,\
    if_test=True, IF_loadmem=opt['IF_savemem_test'])

production_dir = os.path.join(root_dir,'test','production')
tar2_dir = os.path.join(root_dir,'test','aglo_mat')
prodset = myDataset(production_dir,tar2_dir,rgb_dir,transform=dataset_transform,\
    if_test=True, IF_loadmem=opt['IF_savemem_test'])

testloader = DataLoader(prodset, batch_size=opt['test_batch_size'],
                        shuffle=False, num_workers=num_workers)


def test_phase2(opt,net,testloader,log_save_path=None):
    with torch.no_grad():
        net.eval()
        start = time()
        avg_frame_rate = 0
        mae = 0.0
        rmse = 0.0
        me = 0.0
        pre2 = ""
        for j, data in enumerate(testloader):
            inputs, labels = data['image'], data['target']
            inputs, labels = inputs.type(torch.float32),labels.unsqueeze(1).type(torch.float32)
            inputs, labels = inputs.cuda(), labels.cuda()
            pre2 = data['image']
            features = net(inputs)
            div_res = net.resample(features)
            merge_res = net.parse_merge(div_res)
            outputs = merge_res['div'+str(net.args['div_times'])]
            del merge_res
         
            pre =  (outputs).sum()
            gt = labels.sum()
            pre =  int(float(pre))
            print('IMG ' + str(j+1) +":", pre)                                 
            mae += abs(pre-gt)
            rmse += (pre-gt)*(pre-gt)
            me += (pre-gt)
            end = time()
            running_frame_rate = opt['test_batch_size'] * float( 1 / (end - start))
            avg_frame_rate = (avg_frame_rate*j + running_frame_rate)/(j+1)

        if log_save_path:
            txt_write(log_save_path,log_str,mode='w')
    return pre    


test_phase2(opt,net,testloader,log_save_path=log_save_path)
print('======')



IMG 1: 24
IMG 2: 16
IMG 3: 9
IMG 4: 29
IMG 5: 50
